In [1]:
# Execute if necessary
# %%capture
# !pip install numpy seaborn matplotlib pandas

In [2]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from typing import Dict, Tuple, Union, List

# Práctica 3: Modelo Lineal

__Instrucciones__: A continuación hay una lista de funciones que debe implementar o tareas que debe desarrollar. La descripción de cada una de ellas se encuentra en la definición de cada una de las funciones. Cada función está marcada por &#x1F625;,  &#x1F643; o &#x1F921;. Las marcas indican:

- &#x1F625;: Indican una entrega que debe ser hecha dentro de la misma sesión de la asignación. 
- &#x1F643;: Indican una entrega que puede ser hecha hasta la siguiente sesión.
- &#x1F921;: Debe mostrar un avance en la misma sesión, pero la entrega puede ser hecha en la siguiente.

Aquellas entregas parciales que no sean hechas el día de la asignación ya no serán válidas para las entregas totales, sin embargo, las entregas totales seguirán siendo válidas.

En esta sección se incluye un dataset real. El dataset importado incluye multiples características que describen las condiciones de los pasajeros en el accidente del titanic.

- __PassengerId__: Identificador de cada pasajero.
- __Survived__: 0 si no sobrevivió al accidente, 1 si lo hizo.
- __Pclass__: Clase en la que viajaba el pasajero, 1 - Primera clase, 2 - Segunda clase y 3 - Tercera clase.
- __Name__: Nombre del pasajero.
- __Sex__: Sexo del pasajero.
- __Age__: Edad del pasajero.
- __SibSp__: Número de hermanos más número de esposas con las que viajaba el pasajero.
- __Parch__: Número de padres más número de hijos con las que viajaba el pasajero.
- __Ticket__: Número de boleto.
- __Fare__: Tarifa del boleto del pasajero.
- __Cabin__: Número de cabina del pasajero.
- __Embarked__: Puerto de embarcación, C - Cherbourg, Q - Queenstown y S - Southampton.


In [3]:
df = pd.read_csv("titanic.csv")
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


## Asignación 1 &#x1F625;

Realice el preprocesamiento que considere adecuado para que las características __Pclass__, __Sex__, __SibSp__, __Parch__, __Fare__, __Cabin__, __Embarked__ y __Survived__ puedan ser utilizadas por un modelo lineal.

In [4]:
# Creamos un df con solo dichas columnas
data = df.drop(columns = ["PassengerId", "Name", "Age", "Ticket", "Cabin"])
data

,Survived,Pclass,Sex,SibSp,Parch,Fare,Embarked
0,0,3,male,1,0,7.2500,S
1,1,1,female,1,0,71.2833,C
2,1,3,female,0,0,7.9250,S
3,1,1,female,1,0,53.1000,S
4,0,3,male,0,0,8.0500,S
...,...,...,...,...,...,...,...
886,0,2,male,0,0,13.0000,S
887,1,1,female,0,0,30.0000,S
888,0,3,female,1,2,23.4500,S
889,1,1,male,0,0,30.0000,C


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  889 non-null    object 
dtypes: float64(1), int64(4), object(2)
memory usage: 48.9+ KB


In [6]:
data.isnull().sum()

Survived    0
Pclass      0
Sex         0
SibSp       0
Parch       0
Fare        0
Embarked    2
dtype: int64

In [7]:
# Borrar los nulos
data[data["Embarked"].isna()]

,Survived,Pclass,Sex,SibSp,Parch,Fare,Embarked
61,1,1,female,0,0,80.0,NaN
829,1,1,female,0,0,80.0,NaN


In [8]:
# Eliminar nulos
dFD = data.dropna()
dFD.isnull().sum()

Survived    0
Pclass      0
Sex         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [9]:
# Convertir las características categóricas en variables dummy
sex_dummies = pd.get_dummies(dFD['Sex'], prefix='Sex')
embarked_dummies = pd.get_dummies(dFD['Embarked'], prefix='Embarked')
df_selected = pd.concat([dFD, sex_dummies, embarked_dummies], axis=1)
df_selected.drop(['Sex', 'Embarked'], axis=1, inplace=True)

# Normalizar las características numéricas
numeric_features = ['Pclass', 'SibSp', 'Parch', 'Fare']
for feature in numeric_features:
    feature_mean = df_selected[feature].mean()
    feature_std = df_selected[feature].std()
    df_selected[feature] = (df_selected[feature] - feature_mean) / feature_std

print(df_selected)

     Survived    Pclass     SibSp     Parch      Fare  Sex_female  Sex_male  \
0           0  0.824744  0.431108 -0.474059 -0.499958           0         1   
1           1 -1.571327  0.431108 -0.474059  0.788503           1         0   
2           1  0.824744 -0.474932 -0.474059 -0.486376           1         0   
3           1 -1.571327  0.431108 -0.474059  0.422623           1         0   
4           0  0.824744 -0.474932 -0.474059 -0.483861           0         1   
..        ...       ...       ...       ...       ...         ...       ...   
886         0 -0.373291 -0.474932 -0.474059 -0.384258           0         1   
887         1 -1.571327 -0.474932 -0.474059 -0.042189           1         0   
888         0  0.824744  0.431108  2.004991 -0.173986           1         0   
889         1 -1.571327 -0.474932 -0.474059 -0.042189           0         1   
890         0  0.824744 -0.474932 -0.474059 -0.489897           0         1   

     Embarked_C  Embarked_Q  Embarked_S  
0        

In [10]:
# Normalizar

In [11]:
def scale_data(df_selected, cols):
    for c in cols:
        column = df_selected[c]
        sc_column = scale_column(column)
        data[c] = sc_column
    return data

In [12]:
def scale_column(x):
    return (x - x.mean()) / x.std()

In [13]:
df_selected = scale_data(df_selected, ['SibSp', 'Parch', 'Fare'])

In [14]:
df_selected

,Survived,Pclass,Sex,SibSp,Parch,Fare,Embarked
0,0,3,male,0.431108,-0.474059,-0.499958,S
1,1,1,female,0.431108,-0.474059,0.788503,C
2,1,3,female,-0.474932,-0.474059,-0.486376,S
3,1,1,female,0.431108,-0.474059,0.422623,S
4,0,3,male,-0.474932,-0.474059,-0.483861,S
...,...,...,...,...,...,...,...
886,0,2,male,-0.474932,-0.474059,-0.384258,S
887,1,1,female,-0.474932,-0.474059,-0.042189,S
888,0,3,female,0.431108,2.004991,-0.173986,S
889,1,1,male,-0.474932,-0.474059,-0.042189,C


## Asignación 2 &#x1F625;

Utilizando las características __Pclass__, __Sex__, __SibSp__, __Parch__, __Fare__, __Cabin__ y __Embarked__, entrene un clasificador lineal para predecir __Survived__ utilizando el algoritmo _pocket_. Imprima el error obtenido.

In [15]:
# Paso 1: Importar bibliotecas
from sklearn.model_selection import train_test_split

In [16]:
data_ini = df.drop(columns = ["PassengerId", "Name", "Ticket", "Cabin"])
data_ini

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [17]:
data_ini.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [18]:
moda = data_ini["Age"].mode()
print(moda)

0    24.0
Name: Age, dtype: float64


In [19]:
data_ini = data_ini.fillna(24)

In [20]:
data_ini["Sex"] = pd.factorize(data_ini["Sex"])[0]
data_ini["Embarked"] = pd.factorize(data_ini["Embarked"])[0]
data_ini.fillna(data_ini.mean())

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,0
1,1,1,1,38.0,1,0,71.2833,1
2,1,3,1,26.0,0,0,7.9250,0
3,1,1,1,35.0,1,0,53.1000,0
4,0,3,0,35.0,0,0,8.0500,0
...,...,...,...,...,...,...,...,...
886,0,2,0,27.0,0,0,13.0000,0
887,1,1,1,19.0,0,0,30.0000,0
888,0,3,1,24.0,1,2,23.4500,0
889,1,1,0,26.0,0,0,30.0000,1


In [21]:
def scale_column(x):
    return (x - x.mean()) / x.std()

In [22]:
def scale_data(data_ini, cols):
    for c in cols:
        column = data_ini[c]
        sc_column = scale_column(column)
        data_ini[c] = sc_column
    return data_ini

In [23]:
data_ini = scale_data(data_ini, ['SibSp', 'Parch', 'Fare', 'Age'])

In [24]:
data_ini

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,-0.497514,0.432550,-0.473408,-0.502163,0
1,1,1,1,0.714647,0.432550,-0.473408,0.786404,1
2,1,3,1,-0.194474,-0.474279,-0.473408,-0.488580,0
3,1,1,1,0.487367,0.432550,-0.473408,0.420494,0
4,0,3,0,0.487367,-0.474279,-0.473408,-0.486064,0
...,...,...,...,...,...,...,...,...
886,0,2,0,-0.118714,-0.474279,-0.473408,-0.386454,0
887,1,1,1,-0.724794,-0.474279,-0.473408,-0.044356,0
888,0,3,1,-0.345994,0.432550,2.007806,-0.176164,0
889,1,1,0,-0.194474,-0.474279,-0.473408,-0.044356,1


In [25]:
# Paso 4: Separar datos en conjuntos de entrenamiento y prueba
X = data_ini.drop("Survived", axis=1)
y = data_ini["Survived"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [26]:
# Paso 5: Definir función de entrenamiento del clasificador
def train(X_train, y_train, X_test, y_test, max_iter=1000):
    # Agregar una columna de unos para el término de sesgo
    X_train = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
    X_test = np.hstack((np.ones((X_test.shape[0], 1)), X_test))

    # Inicializar pesos aleatorios
    w = np.random.randn(X_train.shape[1])

    # Entrenar el clasificador con el algoritmo pocket
    best_w = w
    best_error = np.inf
    error = np.zeros(max_iter)
    for i in range(max_iter):
        y_pred = np.sign(np.dot(X_train, w))
        y_pred[y_pred == 0] = -1
        error[i] = sum((pred != train for pred, train in zip(y_pred,y_train)))/len(y_pred)
        #np.mean(y_pred != y_train)

        if error[i] < best_error:
            best_w = w.copy()
            best_error = error[i]

        # Actualizar pesos con el algoritmo de perceptrón
        idx = np.where(y_pred != y_train)[0]
        if len(idx) == 0:
            break
        x = X_train[idx[0]]
        y_true = y_train[idx[0]]
        w = w + y_true * x

    # Calcular error en conjunto de prueba
    y_pred_test = np.sign(np.dot(X_test, best_w))
    y_pred_test[y_pred_test == 0] = -1
    test_error = np.mean(y_pred_test != y_test)

    return best_w, best_error, test_error, error

In [27]:
# Paso 6: Entrenar el clasificador y calcular error de entrenamiento y prueba
w, train_error, test_error, error = train(X_train, y_train, X_test, y_test)

# Imprimir resultados
print("Error de entrenamiento:", train_error)
print("Error de prueba:", test_error)

Error de entrenamiento: 0.973314606741573
Error de prueba: 0.9608938547486033


## Asignación 3 &#x1F625;

Utilizando las características __Pclass__, __Sex__, __SibSp__, __Parch__, __Cabin__ y __Embarked__, entrene una regresión lineal para predecir __Fare__ utilizando el algoritmo de Ordinary Leasts Squares (OLS). Imprima el valor del error cuadrático medio (MSE).

In [28]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [29]:
# Selección de características y preprocesamiento de datos
data_pt = df.drop(columns = ["PassengerId", "Survived", "Name", "Age", "Ticket", "Cabin"])

In [30]:
data_pt

,Pclass,Sex,SibSp,Parch,Fare,Embarked
0,3,male,1,0,7.2500,S
1,1,female,1,0,71.2833,C
2,3,female,0,0,7.9250,S
3,1,female,1,0,53.1000,S
4,3,male,0,0,8.0500,S
...,...,...,...,...,...,...
886,2,male,0,0,13.0000,S
887,1,female,0,0,30.0000,S
888,3,female,1,2,23.4500,S
889,1,male,0,0,30.0000,C


In [31]:
data_pt.isnull().sum()

Pclass      0
Sex         0
SibSp       0
Parch       0
Fare        0
Embarked    2
dtype: int64

In [32]:
data_pt = scale_data(data_pt, ['Fare'])
data_pt

,Pclass,Sex,SibSp,Parch,Fare,Embarked
0,3,male,1,0,-0.502163,S
1,1,female,1,0,0.786404,C
2,3,female,0,0,-0.488580,S
3,1,female,1,0,0.420494,S
4,3,male,0,0,-0.486064,S
...,...,...,...,...,...,...
886,2,male,0,0,-0.386454,S
887,1,female,0,0,-0.044356,S
888,3,female,1,2,-0.176164,S
889,1,male,0,0,-0.044356,C


In [33]:
sex_dummies = pd.get_dummies(data_pt['Sex'], prefix='Sex')
embarked_dummies = pd.get_dummies(data_pt['Embarked'], prefix='Embarked')
data_pt = pd.concat([data_pt, sex_dummies, embarked_dummies], axis=1)
data_pt.drop(['Sex', 'Embarked'], axis=1, inplace=True)

numeric_features = ['Pclass', 'SibSp', 'Parch', 'Fare']
for feature in numeric_features:
    feature_mean = data_pt[feature].mean()
    feature_std = data_pt[feature].std()
    data_pt[feature] = (data_pt[feature] - feature_mean) / feature_std

In [34]:
data_pt

,Pclass,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0.826913,0.432550,-0.473408,-0.502163,0,1,0,0,1
1,-1.565228,0.432550,-0.473408,0.786404,1,0,1,0,0
2,0.826913,-0.474279,-0.473408,-0.488580,1,0,0,0,1
3,-1.565228,0.432550,-0.473408,0.420494,1,0,0,0,1
4,0.826913,-0.474279,-0.473408,-0.486064,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...
886,-0.369158,-0.474279,-0.473408,-0.386454,0,1,0,0,1
887,-1.565228,-0.474279,-0.473408,-0.044356,1,0,0,0,1
888,0.826913,0.432550,2.007806,-0.176164,1,0,0,0,1
889,-1.565228,-0.474279,-0.473408,-0.044356,0,1,1,0,0


In [35]:
# División de los datos en conjunto de entrenamiento y prueba
train_size = int(len(data_pt) * 0.8)
train_set = data_pt[:train_size]
test_set = data_pt[train_size:]

X_train = train_set.drop('Fare', axis=1).values
y_train = train_set['Fare'].values
X_test = test_set.drop('Fare', axis=1).values
y_test = test_set['Fare'].values

In [36]:
# Ajuste de la regresión lineal con el algoritmo OLS
X_train = np.c_[np.ones(X_train.shape[0]), X_train]
coefficients = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train

In [37]:
# Evaluación del rendimiento en el conjunto de prueba
X_test = np.c_[np.ones(X_test.shape[0]), X_test]
y_pred = X_test @ coefficients
mse = np.mean((y_test - y_pred) ** 2)

print(f"Error cuadrático medio (MSE): {np.round(mse, 3)}")

Error cuadrático medio (MSE): 0.824


## Asignación 4 &#x1F921;

Utilizando las características __Pclass__, __Sex__, __SibSp__, __Parch__, __Fare__, __Cabin__ y __Embarked__, entrene un clasificador lineal para predecir la probabilidad de supervivencia __Survived__ utilizando el algoritmo de gradiente descendente estocástico y la entropía cruzada como función de error. Imprima el arror en cada iteración del gradiente.

In [38]:
import numpy as np

# Selección de características y preprocesamiento de datos
selected_features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Survived']
data_nny = df[selected_features]

In [39]:
data_nny

,Pclass,Sex,SibSp,Parch,Fare,Embarked,Survived
0,3,male,1,0,7.2500,S,0
1,1,female,1,0,71.2833,C,1
2,3,female,0,0,7.9250,S,1
3,1,female,1,0,53.1000,S,1
4,3,male,0,0,8.0500,S,0
...,...,...,...,...,...,...,...
886,2,male,0,0,13.0000,S,0
887,1,female,0,0,30.0000,S,1
888,3,female,1,2,23.4500,S,0
889,1,male,0,0,30.0000,C,1


In [40]:
# Imputar los valores faltantes en la columna 'Fare'
# fare_median = data_nny['Fare'].median()
# data_nny = data_nny['Fare'].fillna(fare_median)
sex_dummies = pd.get_dummies('Sex')
embarked_dummies = pd.get_dummies('Embarked')

In [41]:
data_nny = pd.concat([data_nny, sex_dummies, embarked_dummies], axis=1)

In [42]:
data_nny

,Pclass,Sex,SibSp,Parch,Fare,Embarked,Survived,Sex,Embarked
0,3,male,1,0,7.2500,S,0,1.0,1.0
1,1,female,1,0,71.2833,C,1,NaN,NaN
2,3,female,0,0,7.9250,S,1,NaN,NaN
3,1,female,1,0,53.1000,S,1,NaN,NaN
4,3,male,0,0,8.0500,S,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
886,2,male,0,0,13.0000,S,0,NaN,NaN
887,1,female,0,0,30.0000,S,1,NaN,NaN
888,3,female,1,2,23.4500,S,0,NaN,NaN
889,1,male,0,0,30.0000,C,1,NaN,NaN


In [43]:
data_nny.isnull().sum()

Pclass        0
Sex           0
SibSp         0
Parch         0
Fare          0
Embarked      2
Survived      0
Sex         890
Embarked    890
dtype: int64

In [44]:
# Selección de características y preprocesamiento de datos
data_nny.drop(columns = ['Sex', 'Embarked'], axis=1, inplace=True)

In [45]:
data_nny

,Pclass,SibSp,Parch,Fare,Survived
0,3,1,0,7.2500,0
1,1,1,0,71.2833,1
2,3,0,0,7.9250,1
3,1,1,0,53.1000,1
4,3,0,0,8.0500,0
...,...,...,...,...,...
886,2,0,0,13.0000,0
887,1,0,0,30.0000,1
888,3,1,2,23.4500,0
889,1,0,0,30.0000,1


In [46]:
data_nny = scale_data(data_nny, ['Fare'])

In [47]:
data_nny

,Pclass,SibSp,Parch,Fare,Survived
0,3,1,0,-0.502163,0
1,1,1,0,0.786404,1
2,3,0,0,-0.488580,1
3,1,1,0,0.420494,1
4,3,0,0,-0.486064,0
...,...,...,...,...,...
886,2,0,0,-0.386454,0
887,1,0,0,-0.044356,1
888,3,1,2,-0.176164,0
889,1,0,0,-0.044356,1


In [48]:
numeric_features = ['Pclass', 'SibSp', 'Parch', 'Fare']
for feature in numeric_features:
    feature_mean = data_nny[feature].mean()
    feature_std = data_nny[feature].std()
    data_nny[feature] = (data_nny[feature] - feature_mean) / feature_std

In [49]:
# Sigmoid
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [50]:
# Entropía cruzada
def cross_entropy_loss(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-8, 1 - 1e-8)
    return -np.mean(y_true*np.log(y_pred) + (1-y_true)*np.log(1-y_pred))

In [51]:
def stochastic_gradient_descent(X_train, y_train, learning_rate=0.001, iterss=100, batch_size=32):

    n_features = X_train.shape[1]
    weights = np.random.randn(n_features)
    biases = np.zeros(1)
    n_batches = len(X_train) // batch_size
    errors = []

    for iters in range(iterss):
        for batch in range(n_batches):

            X_batch = X_train[batch*batch_size : (batch+1)*batch_size]
            y_batch = y_train[batch*batch_size : (batch+1)*batch_size]

            y_pred = sigmoid(np.dot(X_batch, weights) + biases)

            error = cross_entropy_loss(y_batch, y_pred)
            errors.append(error)
            
            dW = np.dot(X_batch.T, (y_pred - y_batch)) / len(X_batch)
            db = np.sum(y_pred - y_batch) / len(X_batch)
            weights -= learning_rate * dW
            biases -= learning_rate * db

        print(f"iters {iters}: Error = {error:.4f}")

    return weights, biases, errors

In [53]:
stochastic_gradient_descent(X, y, 0.001, 100, 32)

iters 0: Error = 1.0825
iters 1: Error = 1.0780
iters 2: Error = 1.0736
iters 3: Error = 1.0692
iters 4: Error = 1.0649
iters 5: Error = 1.0606
iters 6: Error = 1.0563
iters 7: Error = 1.0521
iters 8: Error = 1.0479
iters 9: Error = 1.0437
iters 10: Error = 1.0395
iters 11: Error = 1.0354
iters 12: Error = 1.0313
iters 13: Error = 1.0273
iters 14: Error = 1.0232
iters 15: Error = 1.0192
iters 16: Error = 1.0153
iters 17: Error = 1.0113
iters 18: Error = 1.0074
iters 19: Error = 1.0035
iters 20: Error = 0.9996
iters 21: Error = 0.9957
iters 22: Error = 0.9919
iters 23: Error = 0.9881
iters 24: Error = 0.9843
iters 25: Error = 0.9805
iters 26: Error = 0.9767
iters 27: Error = 0.9730
iters 28: Error = 0.9693
iters 29: Error = 0.9655
iters 30: Error = 0.9619
iters 31: Error = 0.9582
iters 32: Error = 0.9545
iters 33: Error = 0.9509
iters 34: Error = 0.9473
iters 35: Error = 0.9437
iters 36: Error = 0.9401
iters 37: Error = 0.9365
iters 38: Error = 0.9329
iters 39: Error = 0.9294
iters 40: 

(array([-0.71024056,  1.34664649,  1.40171085, -0.35641735, -1.04286445,
        -0.69724187, -0.09949575]),
 array([0.16279445]),
 [1.4754860631801678,
  1.5011280498949529,
  1.6009984637673038,
  1.2086894300383653,
  1.082980820971484,
  1.7297261051918573,
  1.148161299406908,
  1.6185586217148078,
  1.6022539396613917,
  1.9848227790374986,
  1.8920480056594047,
  1.0367414994667412,
  1.0840185929321768,
  1.5482191247621757,
  1.6032077390257775,
  1.3079033234849082,
  1.2257031985199665,
  1.364423600168738,
  1.0203618861924402,
  1.3711565418588862,
  1.0736870936198004,
  1.958998529555308,
  1.120710958105466,
  1.9065182923811066,
  1.2572857742058619,
  1.9325391037498507,
  1.0824858503606611,
  1.4663338553534846,
  1.4932057552685887,
  1.5931785166283465,
  1.204585790721804,
  1.0788650772625803,
  1.7232915759835572,
  1.1424782817368904,
  1.6128432272661974,
  1.5953397476559372,
  1.9725879076681294,
  1.8826024333877933,
  1.0297476155697702,
  1.0797704535094